In [15]:
import pandas as pd


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


In [48]:
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
df1.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)


path = '/home/daisy/FDA_Dataset/inpatient_all_final_2.csv'
df2 = pd.read_csv(path).iloc[:,1:]
df2.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)

In [12]:
df1.shape

(84536, 81)

In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Internalpatientid             84536 non-null  int64  
 1   num_stays                     84536 non-null  int64  
 2   stay_length                   84536 non-null  float64
 3   num_unique_units              84536 non-null  int64  
 4   num_transfers                 84536 non-null  int64  
 5   num_cvd_readmission           84536 non-null  int64  
 6   Readmission                   84536 non-null  int64  
 7   Died                          84536 non-null  int64  
 8   AO                            84536 non-null  int64  
 9   CVD                           84536 non-null  int64  
 10  unique_admitting_specialty    84536 non-null  int64  
 11  unique_discharging_specialty  84536 non-null  int64  
 12  DOMICILIARY                   84536 non-null  int64  
 13  M

### Train test split


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Internalpatientid             84536 non-null  int64  
 1   num_stays                     84536 non-null  int64  
 2   stay_length                   84536 non-null  float64
 3   num_unique_units              84536 non-null  int64  
 4   num_transfers                 84536 non-null  int64  
 5   num_cvd_readmission           84536 non-null  int64  
 6   Readmission                   84536 non-null  int64  
 7   Died                          84536 non-null  int64  
 8   AO                            84536 non-null  int64  
 9   CVD                           84536 non-null  int64  
 10  unique_admitting_specialty    84536 non-null  int64  
 11  unique_discharging_specialty  84536 non-null  int64  
 12  DOMICILIARY                   84536 non-null  int64  
 13  M

In [49]:
X_admission = df1.drop(columns = ['Readmission'])
Y_admission = df1[['Readmission']]

X_mortality = df1.drop(columns = ['Died'])
Y_mortality = df1[['Died']]

In [50]:
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_admission, Y_admission, test_size=0.20, random_state=42)
X_train_mor, X_test_mor, y_train_mor, y_test_mor = train_test_split(X_mortality, Y_mortality, test_size=0.20, random_state=42)


In [ ]:
X_train_ad[X_train_ad.isna().any()]

#### Logistic Regression

In [51]:
lr = LogisticRegression()
# Training
lr.fit(X_train_ad, y_train_ad)

# Prediction
lr_prediction = lr.predict(X_test_ad)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
print(classification_report(y_test_ad, lr_prediction, target_names= ['Not Readmitted', 'Readmitted']))